Distracted Driver Detection - This was originally a competition hosted on kaggle.com consisting of a classification problem in computer vision.

SETTING UP DATA

In [144]:
import csv
d={}
with open("driver_imgs_list.csv") as file_obj:
    file_data = csv.reader(file_obj)
    next(file_data)
    for row in file_data:
        key = row[1].lower()
        if key in d:
            d[key].append(row[2])
        else:
            d[key]=[row[2]]

In [145]:
category_list = list(d.keys())
category_list

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

Our main Task for Setting Up Data is to -
1. Creating Parent Directory as MasterDir and creating 2 Sub Directory Training and Testing 
2. Creating All category folder/directory for both testing and training Sub-folders
3. Copying the Images from imgs/train to MasterDir such that 80% data will go for training and rest for Testing Purpose

In [146]:
# # Creating Parent Directory as MasterDir and creating 2 Sub Directory Training and Testing 
import os
os.mkdir("MasterDir")
os.mkdir("MasterDir/Training")
os.mkdir("MasterDir/Testing")


# Creating All category folder/directory for both testing and training Sub-folders
for each in category_list:
    os.mkdir(os.path.join("MasterDir/Training/",each))
    os.mkdir(os.path.join("MasterDir/Testing/",each))

In [147]:
# Copying the Images from imgs/train to MasterDir such that 80% data will go for training and rest for Testing Purpose
from shutil import copyfile
import os
split_size = 0.8

for category, images in d.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('imgs/train/',category,image)
    dest = os.path.join('MasterDir/Training/', category, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('imgs/train/',category, image)
    dest = os.path.join('MasterDir/Testing/', category, image)
    copyfile(source, dest)

Now we need to get Generator file for our model training which will be done with the Help of Keras.ImageDataGenerator Only

In [148]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
training_dir = "MasterDir/Training/"
testing_dir = "MasterDir/Testing/"

train_datagen=ImageDataGenerator(rescale=0.1/255)
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(64,64),
    class_mode = "categorical",
    batch_size=128
)

test_datagen = ImageDataGenerator(rescale=0.1/255)
test_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size=(64,64),
    class_mode = "categorical",
    batch_size=128
)

Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


So, Here we got 17934 Images with 10 Classes for our Training Data
and 4490 images with 10 Classes for Our Testing Data

BUILDING THE MODEL

In [149]:
# Importing Tensorflow, ImageDataGenerator,CNN_models
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Dropout
from tensorflow.keras.models import Model

In [153]:
# Creating the CNN model
model = tf.keras.models.Sequential()


# Add Convolutional and Pooling layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

# Add fully connected layers
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=Adam(lr=0.01),loss="categorical_crossentropy",metrics=["acc"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 6, 6, 128)       

In [157]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.001)

In [158]:
model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])

C:\Users\visha\AppData\Local\Temp\ipykernel_10016\1505707148.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])


Epoch 1/10
141/141 [==============================] - 124s 877ms/step - loss: 0.6668 - acc: 0.7888 - val_loss: 5.4370 - val_acc: 0.2336
Epoch 2/10
141/141 [==============================] - 75s 531ms/step - loss: 0.1799 - acc: 0.9478 - val_loss: 5.7073 - val_acc: 0.2367
Epoch 3/10
141/141 [==============================] - 72s 512ms/step - loss: 0.0951 - acc: 0.9717 - val_loss: 6.8354 - val_acc: 0.2292
Epoch 4/10
141/141 [==============================] - 71s 504ms/step - loss: 0.0609 - acc: 0.9823 - val_loss: 6.3867 - val_acc: 0.2116


In [1]:
import numpy as np
image = tf.keras.preprocessing.image.load_img("imgs/test/img_100407.jpg",target_size=(64,64,3))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
predictions = model.predict(input_arr)
for i in range(len(predictions[0])):
    if predictions[0][i]==1:
        print(category_list[i])

NameError: name 'tf' is not defined